In [1]:
from wikiparser_utils import WikiXMLDump, WikiPage
import os
import nltk
import json
import numpy as np
from tqdm.notebook import tqdm
from difflib import Differ 
from utils.difflibparser import DifflibParser, DiffCode

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from io import StringIO, BytesIO
from PyPDF2 import PdfReader, PdfFileReader
import urllib3
import html2text
import os
import time
from os import listdir
from os.path import isfile, join

/home/ubuntu/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
from selenium.webdriver.chrome.options import Options
http_urllib = urllib3.PoolManager()
options = Options()
options.headless = True

In [3]:
dump = WikiXMLDump('history.xml')

In [4]:
#dump.get_save_on_disk('pages')

In [5]:
DOCS_DIR = 'data/documents'
DOCS_MAPPER_DIR = 'data/documents_mapper'

if not os.path.exists(DOCS_MAPPER_DIR):
    os.makedirs(DOCS_MAPPER_DIR)
    
if not os.path.exists(DOCS_DIR):
    os.makedirs(DOCS_DIR)
    

In [6]:
dirs = sorted(os.listdir('data/pages'))
stats = {}

In [ ]:
for page_name in tqdm(dirs, total=len(dirs), leave=True, position=0):
    page_docs_mapper_link2id = {}
    page_docs_mapper_id2link = {}
    
    page_revisions = WikiPage().load_revisions(f"data/pages/{page_name}")
    rev_list = sorted(list(map(int, page_revisions)))
    if page_name not in stats:
        stats[page_name] = {}
    stats[page_name]['num of revisions'] = len(rev_list)
    
    last_rev = page_revisions[rev_list[-1]]
    docs_links, _ = last_rev.get_links()
    stats[page_name]['num of document links'] = len(docs_links)
    
    DOCS_WITH_TEXT = 0
    EMPTY_DOCS = 0
    for link_id, link in tqdm(enumerate(docs_links), total=len(docs_links), position=1, leave=False):
        url_link = link[1]
        page_docs_mapper_id2link[link_id] = url_link
        if url_link not in page_docs_mapper_link2id:
            page_docs_mapper_link2id[url_link] = link_id
        
        if not os.path.exists(f'{DOCS_DIR}/{page_name}'):
            os.makedirs(f'{DOCS_DIR}/{page_name}')
        if os.path.exists(f'{DOCS_DIR}/{page_name}/{link_id}.txt'):
            continue
            
        with open(f'{DOCS_DIR}/{page_name}/{link_id}.txt', 'w', encoding='utf-8') as f:
            if 'pdf' == url_link[-3:]:
                try:
                    response = http_urllib.request('GET', url_link)
                    if str(response.status).strip() == '200':
                        memoryFile = BytesIO(response.data)
                        pdfFile = PdfFileReader(memoryFile)
                        answer = ''
                        for pageNum in range(pdfFile.getNumPages()):
                            currentPage = pdfFile.getPage(pageNum)
                            answer += currentPage.extract_text() + '\n'
                        f.write(answer)
                    else:
                        pass
                except:
                    driver = webdriver.Chrome(options=options)
                    driver.get('https://pdfmyurl.com/')
                    a = driver.find_element(By.ID, 'url')
                    a.send_keys(url_link)
                    button_element = driver.find_element(By.NAME, "submit")
                    button_element.click()
                    time.sleep(10)
                    driver.close()

                    
                    pdf_saved_path = ''
                    for f_n in [f for f in listdir('./') if isfile(join('./', f))]:
                        if 'pdf' in f_n:
                            pdf_saved_path = f"./{f_n}"
                            break
                    if pdf_saved_path:
                        reader = PdfReader(pdf_saved_path)
                        text = ""
                        for page in reader.pages:
                            text += page.extract_text() + "\n"
                        f.write(text)
                        os.remove(pdf_saved_path)

            else:
                try:
                    response = requests.get(url_link)
                    if '200' in str(response):
                        text_html = response.text
                        clean_text = html2text.html2text(text_html)
                        f.write(clean_text)
                    else:
                        driver = webdriver.Chrome(options=options)
                        driver.get(url_link)
                        text_html = driver.page_source
                        clean_text = html2text.html2text(text_html)
                        f.write(clean_text)
                        driver.close()
                except:
                    try:
                        driver = webdriver.Chrome(options=options)
                        driver.get(url_link)
                        text_html = driver.page_source
                        clean_text = html2text.html2text(text_html)
                        f.write(clean_text)
                        driver.close()
                    except:
                        pass
        
        with open(f'{DOCS_DIR}/{page_name}/{link_id}.txt', 'r', encoding='utf-8') as f:
            file_text = f.read()
            if file_text:
                DOCS_WITH_TEXT += 1         
            else:
                EMPTY_DOCS += 1
    
    if not os.path.exists(f'{DOCS_MAPPER_DIR}/{page_name}'):
        os.makedirs(f'{DOCS_MAPPER_DIR}/{page_name}')
    json.dump(page_docs_mapper_id2link, open(f'{DOCS_MAPPER_DIR}/{page_name}/id2link.json', "w"))
    json.dump(page_docs_mapper_link2id, open(f'{DOCS_MAPPER_DIR}/{page_name}/link2id.json', "w"))
    stats[page_name]['num of empty documents'] = EMPTY_DOCS
    stats[page_name]['num of text documents'] = DOCS_WITH_TEXT

  0%|          | 0/925 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]